In [ ]:
import numpy as np
from matplotlib import pyplot as plt

# Task 2.1

In [ ]:
x = np.random.randint(100, size=(100))
x = (x - np.min(x))/np.ptp(x)
x = x.astype(np.float32)
print(x)

In [ ]:
def function(x_):
    return x_**3-x_**2
    
myfunc_vec = np.vectorize(function)
t = myfunc_vec(x)

print(t)

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=4, figsize=(10,5))
plt.tight_layout()
axes[0].plot(np.linspace(0,1,100))
axes[1].plot(myfunc_vec(np.arange(0,100)))
axes[2].plot(x)
axes[3].plot(t)

# Task 2.2

In [ ]:
class Layer:
    def __init__(self, input_units, n_units):
        self.weights = np.random.randn(input_units, n_units)
        self.bias    = np.zeros((1, n_units))

    def forward_step(self, x):
    
        self.input = x
        
        # save calculated outout
        self.preactivation = self.input @ self.weights + self.bias # output
        
        # apply relu
        self.activation = np.maximum(self.preactivation, 0)
        
        return self.activation

    def backward_step(self,in_gradient):
        
        # print shapes
#         print(f"Weights: {self.input.T.shape} @ ({self.preactivation.shape} * {in_gradient.T.shape}")
#         print(f"Bias: {self.preactivation.shape} * {in_gradient.T.shape}")"
#         print(f"Input_grads: {self.preactivation.shape} * {in_gradient.T.shape} @ {self.weights.T.shape}")
        
        #ReLU_deriv = 1 * (self.preactivation > 0) # Why doesnt this work
        ReLU_deriv = self.preactivation
    
        # get gradient wrt. weights
        weight_grads = self.input.T @ (ReLU_deriv * in_gradient)
        
        # get gradient wrt. biases
        bias_grads = (ReLU_deriv * in_gradient)
        
        # get gradient wrt. input
        input_grads = (ReLU_deriv * in_gradient) @ self.weights.T   

        # update step
        self.weights = self.weights - 0.2 * weight_grads
        self.bias = self.bias - 0.2 * bias_grads
        
        return input_grads

In [ ]:
class Model:
    def __init__(self, components) -> None:
        self.components = components

    def forward_step(self, x):
    
        out = self.components[1].forward_step(x)
        out_final = self.components[0].forward_step(out)
        
        return out_final
        
    def backward(self):
        
        final_layer_grad = (self.y_pred-self.y_true)
        #print(f"MSE Grad: {final_layer_grad}")
        layer2_grad = self.components[0].backward_step(final_layer_grad)
        #print(f"Layer2 Grad: {layer2_grad}")
        layer1_grad = self.components[1].backward_step(layer2_grad)
    
    def loss(self,y_true,y_pred):
        
        self.y_true = y_true
        self.y_pred = y_pred
        
        return 0.5*(y_pred-y_true)**2
    

In [ ]:
layer1 = Layer(1,10)
layer2 = Layer(10,1)

m = Model([layer2, layer1])

In [ ]:
loss = []

for epoch in range (0,1000):
    
    running_loss = 0
    
    for idx, datapoint in enumerate(x):        
        
        #print(f"sample: {np.broadcast_to(datapoint,(1,))}")
        #print(f"Real: {np.broadcast_to(t[idx],(1,))}")
        
        # fwp
        out = m.forward_step(np.broadcast_to(datapoint,(1,)))
        
        # update loss
        running_loss += m.loss(t[idx],out)
        
        # bwp
        m.backward()
         
    loss.append((running_loss/len(x))[0])
    print(f"Loss in Epoch {epoch+1}: {(running_loss/len(x))[0]}")
    #break
    
print("Training finished")

In [ ]:
plt.plot(loss)
plt.title("Average Loss per Epoch")

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(10,5))
plt.tight_layout()

plot = []

for point in np.linspace(0,1,100):
    out = m.forward_step(np.broadcast_to(point,(1,)))
    plot.append(out[0])
    
axes[0].plot(np.linspace(0,1,100))
axes[0].set_title("Input")
axes[0].set_ylim(0,1)
axes[1].plot(plot)
axes[1].set_title("Prediction")
axes[1].set_ylim(0,1000000)
axes[2].plot(myfunc_vec(np.arange(0,100)))
axes[2].set_title("True Output")
axes[2].set_ylim(0,1000000)